## 🐶🐱 **Clasificador de Imágenes: Perros vs Gatos**
- hecho por: John Fredy Beltran Cuellar
- Fecha: 11/22/2025
- Meta: 


In [ ]:
# step 0. cargar librerias
import os
import random
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
# step 1 cargar datos
train_path = r"C:/Users/jhonf/Clasificador_de_imagenes_Perros_y_Gatos/dogs-vs-cats/dogs-vs-cats/train"
val_path = r"C:/Users/jhonf/Clasificador_de_imagenes_Perros_y_Gatos/dogs-vs-cats/dogs-vs-cats/validation"

datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

val_data = datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)


In [ ]:
# visualizar 9 imagenes de cada clase
def plot_images(data, label_index, title):
    images = []
    labels = []
    count = 0
    for img, label in data:
        for i in range(len(label)):
            if label[i][label_index] == 1:
                images.append(img[i])
                labels.append(label[i])
                count += 1
                if count == 9:
                    break
        if count == 9:
            break

    plt.figure(figsize=(10, 10))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.axis("off")
    plt.suptitle(title)
    plt.show()

plot_images(train_data, label_index=0, title="Gatos")
plot_images(train_data, label_index=1, title="Perros")


In [ ]:
# construir el modelo CNN tipo VGG16

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # Para clasificación binaria con categorical labels
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
early_stop = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", monitor="val_accuracy", save_best_only=True)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=[early_stop, checkpoint]
)


In [ ]:
# evaluacion y visualizacion
plt.figure(figsize=(10, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train")
plt.plot(history.history["val_accuracy"], label="Validation")
plt.title("Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train")
plt.plot(history.history["val_loss"], label="Validation")
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()



In [ ]:
# guardar el modelo final
model.save("dogs_vs_cats_final_model.h5")
